# Практична робота №1. Отримання навичок роботи в середовищi Python
Варіант 25 \
Виконали студенти групи КА-35 Бернацький Артем, Бубало Анна, Іванкевич Богдан, Шальман Тимофій

### Ініціалізація потрібних змінних та значень

In [1]:
I = {"молоко", "вода", "борошно", "яблука"}
D = [["молоко", "вода", "борошно"],
    ["молоко", "борошно"],
    ["молоко", "вода", "борошно","яблука"],
    ["борошно","вода"],
    ["яблука"]]

supp_min = 0.4

### Реалізація функції $Supp$
$Supp(F) = \frac{|D_F|}{|D|}$

In [2]:
def supp(F, D):
    count = 0
    for row in D:
        if all(i in row for i in F):
            count += 1
    return count / len(D)

### Розрахунок $L_1$
$L_1 = \{ i|i \in{I}, Supp(i) \ge Supp_{min} \}$

In [3]:
L1 = [[i] for i in I if supp([i], D) >= supp_min]
L1

[['вода'], ['яблука'], ['борошно'], ['молоко']]

### Реалізація алгоритму знаходження результуючої множини частих наборiв $L_k$
$L_k=\{ F \cup{i} | F \in{L_{k-1}}, i \in{L_1} \setminus F, Supp(F \cup{ \{i\} }) \ge Supp_{min} \}$

In [4]:
def get_frequent_sets(D, L1, supp_min):
    result = [(supp(F, D), F) for F in L1]
    L_prev = L1.copy()

    while True:
        Lk = []
        for F in L_prev:
            for i in L1:
                item = i[0]
                if item not in F:
                    F_new = sorted(F + [item])
                    if F_new not in Lk:
                        supp_f = supp(F_new, D)
                        if supp_f >= supp_min:
                            Lk.append(F_new)
                            result.append((supp_f, F_new))
        if not Lk:
            break

        L_prev = Lk

    return result

### Результат виконання алгоритму на нашому датасеті

In [5]:
frequent_sets = get_frequent_sets(D, L1, supp_min)
print(" Supp\tSet")
frequent_sets

 Supp	Set


[(0.6, ['вода']),
 (0.4, ['яблука']),
 (0.8, ['борошно']),
 (0.6, ['молоко']),
 (0.6, ['борошно', 'вода']),
 (0.4, ['вода', 'молоко']),
 (0.6, ['борошно', 'молоко']),
 (0.4, ['борошно', 'вода', 'молоко'])]